# Output, Results and Discussions

In order to predict heart disease in patients based on the different health indicators provided in the dataset, we decided to try two different models, `LogisticRegression` and `SVC` (support vector classifier), with default parameters as well as the `DummyClassifier` as a base model for comparison. We conducted 5-fold cross validation on the train set and extracted the mean fit time, score time, test score and train score for each model as well as the standard deviations to compare the models and select one for hyperparameter optimization. We used the f1 score as opposed to accuracy as our scoring metric due to the class imbalance we observed during our EDA. The results are listed in the table below:

In [1]:
import pandas as pd

In [12]:
pd.read_csv("../../results/model_selection_results.csv", index_col=0, header = 0, names=["Dummy: Mean", "Dummy: Std", "SVC: Mean", "SVC: Std", "LogisticRegression: Mean", "LogisticRegression: Std"])

,Dummy: Mean,Dummy: Std,SVC: Mean,SVC: Std,LogisticRegression: Mean,LogisticRegression: Std
fit_time,0.001,0.001,0.008,0.001,0.012,0.001
score_time,0.001,0.000,0.006,0.002,0.005,0.001
test_score,0.726,0.007,0.870,0.038,0.836,0.012
train_score,0.726,0.002,0.908,0.003,0.885,0.008


Table 1: Cross validation mean scores and standard deviation by model

Based on these results, we could see that the `DummyClassifier` model already performs quite well at predicting the presence of disease. However, both `SVC` and `LogisticRegession` had higher mean cross validation scores and were already performing better than the `DummyClassifier` with default values. 

## Conclusions

## Limitations & Future Work
